# logic stuff

Here we tried to automate the following things, which we typically do in logic
- check if a sentence is well-formed
- convert between infix and prefix notation
- check for errors in a model
- assess whether a sentence is true or false in a model
- draw a tree for a sentence, and read a model off an open tree
- do various tree tests

For the theory, see N.J.J. Smith (2012). <i>Logic: The Laws of Truth</i>. PUP

The language here is first-order general predicate logic with identity<br>

<b>Notation</b><br>
= for identity<br>
~a=b for non-identity<br>
~ for negation<br>
& or ^ for conjunction<br>
| for disjunction<br>
-> or > for conditional<br>
<-> or < for biconditional<br>
(Ax) and (Ex) for quantifiers <br>
whitespace is ignored

<b>Future work</b><br>
If we continue working on this, we might try to implement translation from English to GPLI, or make a GUI

### Functions

Here are some functions that make the code work, if you want to help with debugging the code you'll probably want to look here. If not, just shift-enter this cell to get to the main thing

In [ ]:
### Functions for processing strings

# Process user input: remove spaces and shorten all connectives to length 1
def pre(str):
    str = str.replace(" ","")
    str = str.replace("<->","<")
    str = str.replace("->",">")
    str = str.replace("^","&")
    return str

# Spell out shortened connectives for display
def post(str):
    str = str.replace(">","->")
    str = str.replace("<","<->")
    str = str.replace("&","^")
    return str


### Functions that check and convert wffs

# check whether an infix string is a wff
# this function assumes outermost brackets for all binary connectives
# but later the program will not require that from user
def iswff(str):
    # base cases: PL propositions, identity, GPL atoms
    if (str == ""): return False
    if (len(str) == 1): return str.isupper()
    if ((len(str) == 3) and str[0].islower() and str[-1].islower() and (str[1]=="=")): return True
    if (str[0].isupper() and str[1:].isalpha() and str[1:].islower()): return True
    
    # negation
    if (str[0] == '~' and iswff(str[1:])): return True
    
    # quantifiers
    if (len(str)>=4):
        if ((str[:2] in {"(A","(E"}) and (str[3]==")") and str[2].islower() and str[2].isalpha() and iswff(str[4:])): return True
    
    # binary connectives 
    # remove outermost brackets
    if ((str[0] != "(") or (str[-1] != ")")): return False
    str = str[1:-1]
    
    # parse binary
    i = 1
    while (i<(len(str)-1)):
        if (iswff(str[0:i]) and (str[i] in {">","&","|","<"}) and iswff(str[i+1:])): return True
        i += 1
    return False

# convert a wff from infix to prefix
def inToPre(str):
    # base cases
    if (len(str) == 1): return str
    if ((len(str) == 3) and str[0].islower() and str[-1].islower() and (str[1]=="=")): return "=" + str[0] + str[2]
    if (str[0].isupper() and str[1:].isalpha() and str[1:].islower()): return str
    
    # negation
    if (str[0] == '~' and iswff(str[1:])): return "~"+inToPre(str[1:])
    
    # quantifiers
    if (len(str)>=4):
        if ((str[:2] in {"(A","(E"}) and (str[3]==")") and str[2].islower() and str[2].isalpha() and iswff(str[4:])): return str[:4] + inToPre(str[4:])
    
    # outermost brackets
    if iswff(str): str = str[1:-1]
    
    # parse binary
    i = 0
    while not (iswff(str[:i]) and (str[i] in {">","&","|","<"})): i+=1
    return str[i] + inToPre(str[:i]) + inToPre(str[i+1:])

# check if a string in prefix is well-formed
def iswffP(str):
    # base cases
    if (str == ""): return False
    if (len(str) == 1): return str.isupper()
    if ((len(str) == 3) and str[1].islower() and str[-1].islower() and (str[0]=="=")): return True
    if (str[0].isupper() and str[1:].isalpha() and str[1:].islower()): return True
    
    # negation
    if (str[0] == '~'): return iswffP(str[1:])
    
    # quantifiers
    if (len(str)>=4):
        if ((str[:2] in {"(A","(E"}) and (str[3]==")") and str[2].islower() and str[2].isalpha()): return iswffP(str[4:])
    
    # binary connectives first char is the connective
    if (str[0] not in {">","&","|","<"}): return False
    
    # remainder divides into two wffs
    i = 2
    while (i<(len(str)-1)):
        if (iswffP(str[1:i]) and iswffP(str[i:])): return True
        i += 1
    return False

# convert a string from prefix to infix
def preToIn(str):
    # base cases
    if (len(str) == 1): return str
    if ((len(str) == 3) and str[1].islower() and str[-1].islower() and (str[0]=="=")): return str[1] + '=' + str[-1]
    if (str[0].isupper() and str[1:].isalpha() and str[1:].islower()): return str
    
    # negation
    if (str[0] == '~'): return "~"+preToIn(str[1:])
    
    # quantifiers
    if (len(str)>=4):
        if ((str[:2] in {"(A","(E"}) and (str[3]==")") and str[2].islower() and str[2].isalpha() and iswffP(str[4:])): return str[:4] + preToIn(str[4:])
    
    # parse binary
    i = 2
    while not (iswffP(str[1:i]) and iswffP(str[i:])): i+=1
    return '(' + preToIn(str[1:i]) + str[0] + preToIn(str[i:]) + ')'


### functions regarding models
# ask user to input a model
def modin():
    try: 
        # PL propositions, user puts a group of propositions separated by commas
        propstr = input('basic propositions: ')
        prop = propstr.replace(' ','').split(',')
        
        # domain, user inputs objects separated by commas
        domstr = input('domain: ')
        dom = set(domstr.replace(' ','').split(','))
        
        # referents, user inputs [name]: [referent] pairs separated by commas
        refstr = input('referents: ')
        refstr = refstr.replace(' ','')
        
        # referents section may be empty
        if not refstr: ref = {}
        else:
            refstr = refstr.split(',')
            refstr = [i.split(':') for i in refstr]
            # referents stored in a dict
            ref = {}
            for i in refstr: ref[i[0]]=i[1]

        # extensions, user inputs [predicate]: [extension] pairs separated by commas
        # [extension] is a list of lists
        extstr = input('extensions: ')
        extstr = extstr.replace(' ','')
        
        # can be empty
        if not extstr: ext = {}
        else:
            # split into entries
            extstr = extstr.split('},')
            
            # split each entry
            extstr = [i.split(':') for i in extstr]
            
            # now each member of extstr looks like ['F', '{<a,b>,c,d'], might have } if last member
            ext = {}
            
            for i in extstr:
                # empty set
                if (i[1] == '{}' or i[1] == '{'): ext[i[0]] = []; continue
                    
                # remove {}
                if (i[1][-1]=='}'): temp = i[1][1:-1]
                else: temp = i[1][1:]
                    
                # temp is now <a,b>,c,d
                # check if temp is not unary
                if (temp[0] == '<'):
                    # not unary, split into members
                    temp = temp.split('>,')
                    # now each member of temp looks like '<a,b', might have > if last member
                    
                    mem = []
                    
                    # parse each member of temp into a list
                    for j in temp:
                        if (j[-1] == '>'): mem.append(j[1:-1].split(','))
                        else: mem.append(j[1:].split(','))                    
                    ext[i[0]] = mem
                    
                # unary, each member forms a 1-member list
                else: ext[i[0]] = [[k] for k in temp.split(',')]
        
    # if any error, user is asked to check inputs
    except: raise Exception('check inputs')
    return [dom, ref, ext, prop]


# check if a model passes basic tests
# if verbose set to true, this code echoes the errors found if any
def modcheck(model, verbose = True):
    # if no problems, ok
    ok = True
    
    # check basic propositions, must be single upper-case letters
    badprops = set([i for i in model[3] if not (i.isupper() and len(i) == 1)])
    if badprops and (badprops != {''}):
        if verbose: print('proposition symbols not uppercase letters:', badprops)
        ok = False
    
    # check if domain is empty
    if (model[0] == {''}): 
        if verbose: print('domain can\'t be empty')
        ok = False
    
    # check if names are in bad format, or if referents are not in domain
    badnames = set([i for i in model[1].keys() if not (i.islower() and len(i)==1)])
    badrefs = set([i for i in model[1].values() if i not in model[0]])
    
    if badnames: 
        if verbose: print('names not lowercase letters:', badnames)
        ok = False
    if badrefs: 
        if verbose: print('referents not in domain:', badrefs)
        ok = False
    
    # check if predicates are in bad format
    badpreds = set([i for i in model[2].keys() if not (i.isupper() and len(i)==1)])
    # check if extensions are not subsets of domain
    badexts = set()
    # check for inconsistent arity
    badar = set()
    
    for i,j in model[2].items():
        if (j == []): continue
            
        arity = len(j[0])
        
        for k in j:
            if (len(k) != arity): badar.add(i)
            badexts = badexts.union(set([l for l in k if l not in model[0]]))
    
    if badpreds: 
        if verbose: print('predicates not uppercase letters:', badpreds)
        ok = False
    if badexts: 
        if verbose: print('extension members not in domain:', badexts)
        ok = False
    if badar: 
        if verbose: print('inconsistent arity: ', badar)
        ok = False
        
    return ok


# evaluate sentence in model
# if verbose set to true, this code also gives the explanation
# short-circuiting is implemented for brevity in answer
def eval(str, model, verbose = True, indent = 0):
    preprint = '    '*indent
    # base case: proposition
    if (len(str) == 1): 
        result = (str in model[3])
        if verbose: print(preprint + f'The basic proposition {post(str)} is {result}')
        return result
    
    # base case: identity
    if ((len(str) == 3) and (str[1]=="=")): 
        result = (model[1][str[0]] == model[1][str[2]])
        if verbose:
            equal = 'equal' if result else 'not equal'
            print(preprint + f'{post(str)} is {result} because the referent of {str[0]} is {equal} to the referent of {str[2]}')
        return result
    
    # base case: atomic
    if str.isalpha(): 
        l = [model[1][i] for i in list(str[1:])]
        result = (l in model[2][str[0]])
        if verbose:
            equal = 'in' if result else 'not in'
            print(preprint + f'{post(str)} is {result} because {l} is {equal} the extension of {str[0]}')
        return result
    
    # negation
    if (str[0] == '~'): 
        result = eval(str[1:], model, verbose = False)
        if verbose: print(preprint + f'{post(str)} is {not result} because {post(str[1:])} is {result}')
        eval(str[1:], model, verbose = verbose, indent = indent)
        return not result
    
    # for below, if verbose, the formula will be evaluated twice
    # first time with verbose = False to get the result
    # second time with verbose = True to print the most efficient explanation
    # if not verbose, just once to get the result
    
    # quantifiers
    if (len(str)>=4):
        subpreprint = '    '*(indent+1)
        # universal
        if ((str[:2] == "(A") and iswff(str[4:])): 
            # go through each object in the domain
            for object in model[0]:
                # strip quantifier
                tempsen = str[4:]
                
                # add temporary name and referent
                model[1][str[2]] = object
                
                # if found to be false
                if not eval(tempsen, model, False):
                    if verbose:
                        print(preprint + f'{post(str)} is false')
                        print(preprint + f'When {str[2]} is {object}:')
                        _ = eval(tempsen, model, indent = indent+1)
                    return False
            
            # found to be true: if verbose, run again to get explanations
            if verbose: 
                print(preprint + f'{post(str)} is true because {post(tempsen)} is true under all substitutions of {str[2]}')
                for object in model[0]:
                    print(subpreprint + f'When {str[2]} is {object}...')
                    tempsen = str[4:]
                    model[1][str[2]] = object
                    eval(tempsen, model, indent = indent+2)
            return True
        
        # existential
        if ((str[:2] == "(E") and iswff(str[4:])): 
            # go through each object in the domain
            for object in model[0]:
                # strip quantifier
                tempsen = str[4:]
                
                # add temporary name and referent
                model[1][str[2]] = object
                
                # if found to be true
                if eval(tempsen, model, False):
                    if verbose:
                        print(preprint + f'{str} is true')
                        print(preprint + f'When {str[2]} is {object}:')
                        eval(tempsen, model, verbose = verbose, indent = indent+1)
                    return True
            if verbose: 
                print(preprint + f'{post(str)} is false because {post(tempsen)} is false under all substitutions of {str[2]}')
                for object in model[0]:
                    print(subpreprint + f'When {str[2]} is {object}...')
                    tempsen = str[4:]
                    model[1][str[2]] = object
                    eval(tempsen, model, indent = indent+2)
            return False        
        
    # outermost brackets
    if iswff(str): str = str[1:-1]
    
    # parse binary
    i = 0
    while not (iswff(str[:i]) and (str[i] in {">","&","|","<"})): i+=1
    # here str[i] is the connective, str[:i] is left, and str[i+1:] is right
    
    # conjunction
    if (str[i] == '&'):
        resultL = eval(str[:i], model, False)
        resultR = eval(str[i+1:], model, False)
        
        if not verbose: return (resultL and resultR)
        
        # if L is false
        if not resultL:
            eval(str[:i], model, indent = indent)
            print(preprint + f'{post(str)} is false because left conjunct is false')
            return False
        
        # if R is false
        if not resultR:
            eval(str[i+1:], model, indent = indent)
            print(preprint + f'{post(str)} is false because right conjunct is false')
            return False
        
        eval(str[:i], model, indent = indent)
        eval(str[i+1:], model, indent = indent)
        print(preprint + f'{post(str)} is true because both conjuncts are true')
        return True
    
    # disjunction
    if (str[i] == '|'):
        resultL = eval(str[:i], model, False)
        resultR = eval(str[i+1:], model, False)
        
        if not verbose: return (resultL or resultR)
        
        # if L is true
        if resultL:
            eval(str[:i], model, indent = indent)
            print(preprint + f'{post(str)} is true because left disjunct is true')
            return True
        
        # if R is true
        if resultR:
            eval(str[i+1:], model, indent = indent)
            print(preprint + f'{post(str)} is true because right disjunct is true')
            return True
        
        eval(str[:i], model, indent = indent)
        eval(str[i+1:], model, indent = indent)
        print(preprint + f'{post(str)} is false because both disjuncts are false')
        return False
    
    # conditional
    if (str[i] == '>'):
        resultL = eval(str[:i], model, False)
        resultR = eval(str[i+1:], model, False)
        
        if not verbose: return ((not resultL) or resultR)
        
        # if L is false
        if not resultL:
            eval(str[:i], model, indent = indent)
            print(preprint + f'{post(str)} is true because antecedent is false')
            return True
        
        # if R is true
        if resultR:
            eval(str[i+1:], model, indent = indent)
            print(preprint + f'{post(str)} is true because consequent is true')
            return True
        
        eval(str[:i], model, indent = indent)
        eval(str[i+1:], model, indent = indent)
        print(preprint + f'{post(str)} is false because antecedent is true and consequent is false')
        return False
    
    # biconditional
    if (str[i] == '<'):
        resultL = eval(str[:i], model, False)
        resultR = eval(str[i+1:], model, False)
        
        if verbose: 
            eval(str[:i], model, indent = indent)
            eval(str[i+1:], model, indent = indent)
            equal = 'the same truth value' if (resultL == resultR) else 'different truth values'
            print(preprint + f'{post(str)} is true because {post(str[:i])} and {post(str[i+1:])} have {equal}')
        return (resultL == resultR)

    
### functions involving trees

# we'll need deepcopy to implement branching
import copy

# the main connective of a sentence will be encoded as follows to identify the tree rule
# 0: atomic
# 1: basic negation
# 2: identity
# 3: dbl negation
# 4: negated disjunction
# 5: conjunction
# 6: negated conditional
# 7: disjunction
# 8: negated conjunction
# 9: conditional
# 10: biconditional
# 11: negated biconditional
# 12: negated existential
# 13: negated universal
# 14: existential
# 15: universal
# 16: proposition
# 17: negated proposition

# this first function will encode a string
# input is assumed to be a wff
# output is a 3-member list
# 1st member is the string
# 2nd member is the corresponding code
# 3rd member is additional info about the connective
# if there is nothing to be done (codes 0,1,2,16,17), set to 0
# for dbl negation, negated quantifiers, and existential, 1 if not done, 0 if done
# for binary connectives (4-11), length of first wff
# for universal, track substitutions
def labelsen(str):
    # base cases
    if (len(str) == 1): return [str,16,0]
    if (str.isalpha()): return [str,0,0]
    if ((len(str) == 3) and (str[1]=="=")): return [str,2,0]
    
    # negation
    if (str[0] == '~' and iswff(str[1:])): 
        temp = str[1:]
        if (len(temp) == 1): return [str,17,0]
        if temp.isalpha(): return [str,1,0]
        if ((len(temp) == 3) and (temp[1]=="=")): return [str,1,0]
        if (temp[0] == '~' and iswff(temp[1:])): return [str,3,1]

        if (len(temp)>=4):
            if (temp[:2] == "(A" and temp[3]==")" and iswff(temp[4:])): return [str,13,1]
            if (temp[:2] == "(E" and temp[3]==")" and iswff(temp[4:])): return [str,12,1]
        
        # outermost brackets
        if iswff(temp): temp = temp[1:-1]
            
        # parse binary
        i = 0
        while not (iswff(temp[:i]) and (temp[i] in {">","&","|","<"})): i+=1
        if (temp[i] == '>'): return [str,6,i]
        if (temp[i] == '&'): return [str,8,i]
        if (temp[i] == '|'): return [str,4,i]
        if (temp[i] == '<'): return [str,11,i]
        
    # quantifiers
    if (len(str)>=4):
        if (str[:2] == "(A" and str[3]==")" and iswff(str[4:])): return [str,15,set()]
        if (str[:2] == "(E" and str[3]==")" and iswff(str[4:])): return [str,14,1]
    
    # outermost brackets
    if iswff(str): str = str[1:-1]
    
    # parse binary
    i = 0
    while not (iswff(str[:i]) and (str[i] in {">","&","|","<"})): i+=1
    if (str[i] == '>'): return [str,9,i]
    if (str[i] == '&'): return [str,5,i]
    if (str[i] == '|'): return [str,7,i]
    if (str[i] == '<'): return [str,10,i]

    
# create an unlabelled preprocessed sentence by asking user for input
def senIn(prompt):
    try:
        sen = input(prompt+' :')
        sen = pre(sen)
        if not (iswff(sen) or iswff("("+sen+")")): print('not well-formed'); return []
        return sen
    except: raise Exception('check input')

# create a list of labelled sentences from multiple inputs
def labSetIn(prompt):
    try:
        root = []
        sen = input('first ' + prompt + ': ')
        while True:
            sen = pre(sen)
            if not (iswff(sen) or iswff("("+sen+")")): print('not well-formed, sentence not inserted')
            else: root.append(labelsen(sen))
            sen = input('Next ' + prompt + ' (leave blank if done): ')
            if not sen: break
        return root
    except: raise Exception('check input')


# get the set of all names in a sentence
def namesinsen(str):
    # base cases
    if (len(str) == 1): return set()
    if str.isalpha(): return set([char for char in str[1:]])
    if ((len(str) == 3) and (str[1]=="=")): return {str[0],str[2]}
    
    # negation
    if (str[0] == '~' and iswff(str[1:])): return namesinsen(str[1:])
        
    # quantifiers
    if (len(str)>=4):
        if ((str[:2] in {"(A","(E"}) and (str[3]==")")): 
            temp = namesinsen(str[4:])
            if (str[2] in temp): temp.remove(str[2])
            return set(temp)
    
    # outermost brackets
    if iswff(str): str = str[1:-1]
    
    # parse binary
    i = 0
    while not (iswff(str[:i]) and (str[i] in {">","&","|","<"})): i+=1
    return namesinsen(str[:i]).union(namesinsen(str[i+1:]))

# get the set of all names on a branch
def namesinbr(br):
    names = set()
    for entry in br: names = names.union(namesinsen(entry[0]))
    return names

# get the next available name for a branch
def nextname(br):
    alphabet = 'abcedfghijklmnopqrstuvwxyz'
    existing = namesinbr(br)
    
    i = 0
    while (alphabet[i] in existing and i<26): i+=1
    return alphabet[i]
    

# check if branch is open
def checkopen(br):
    allnames = namesinbr(br)
    
    for formula in br:
        # other than universal, all done
        if (formula[1] != 15 and formula[2]): return False
        
        # all names into all universals
        if (formula[1] == 15 and formula[2] != allnames): return False
        
        # SI
        # collect all atomic formulas already present
        atoms = set([formula[0] for formula in br if formula[1] in {0,1,2}])
        # collect all identity sentences already present
        idens = set([formula[0] for formula in br if formula[1] == 2])
            
        # check that no instance of SI results in a new sentence
        for iden in idens:
            for atom in atoms:
                if iden[0] in atom: 
                    newguy = atom.replace(iden[0],iden[2],1)
                    if newguy not in atoms: return False
    
                if iden[2] in atom: 
                    newguy = atom.replace(iden[2],iden[0],1)
                    if newguy not in atoms: return False
    
    # if all good, mark branch as open
    br.append('OPEN BRANCH')
    return True

# check if branch is closed
def checkclosed(br):   
    atomic = set()
    
    # find all positive atomics, identities, and propositions
    for formula in br:
        if (formula[1] == 0 or formula[1] == 2 or formula[1] == 16): atomic.add(formula[0])
    
    # check all basic negations and negations of propositions
    for formula in br:
        if ((formula[1] == 1 or formula[1] == 17) and formula[0][1:] in atomic): br.append('CLOSED BRANCH'); return True
    
    return False

    
# finish tree
def finish(tr):
    # loop over all branches (more branches may be added)
    for br in tr:
        while True:
            # check for open/closed on each loop
            if (checkclosed(br) or checkopen(br)): break
            
            # for efficiency, we'll do in this order
            # non-branching PL stuff
            # negated quantifier and existential quantifier
            # universals
            # branching PL
            # SI
            didsomething = 0
            
            # search the formulas for non-branching
            for formula in br:
                # if done this formula, next
                if (formula[2] == 0): continue
                
                # double negation
                if (formula[1] == 3): 
                    formula[2] = 0
                    br.append(labelsen(formula[0][2:]))
                    didsomething = 1
                    break
                
                # negated disjunction
                if (formula[1] == 4): 
                    br.append(labelsen('~'+formula[0][2:2+formula[2]]))
                    br.append(labelsen('~'+formula[0][3+formula[2]:-1]))
                    formula[2] = 0
                    didsomething = 1
                    break
                    
                # conjunction
                if (formula[1] == 5): 
                    br.append(labelsen(formula[0][:formula[2]]))
                    br.append(labelsen(formula[0][1+formula[2]:]))
                    formula[2] = 0
                    didsomething = 1
                    break
                    
                # negated conditional
                if (formula[1] == 6): 
                    br.append(labelsen(formula[0][2:2+formula[2]]))
                    br.append(labelsen('~'+formula[0][3+formula[2]:-1]))
                    formula[2] = 0
                    didsomething = 1
                    break
                    
            if didsomething: continue
            
            # search the formulas for negated quantifier or existential
            for formula in br:
                # if done this formula, next
                if (formula[2] == 0): continue
                
                # negated universal
                if (formula[1] == 13): 
                    formula[2] = 0
                    br.append(labelsen('(E' + formula[0][3] + ')~' + formula[0][5:]))
                    didsomething = 1
                    break
                
                # negated existential
                if (formula[1] == 12): 
                    formula[2] = 0
                    br.append(labelsen('(A' + formula[0][3] + ')~' + formula[0][5:]))
                    didsomething = 1
                    break
                    
                # existential
                if (formula[1] == 14): 
                    # strip quantifier
                    temp = formula[0][4:]
                    # new name
                    newname = nextname(br)
                    # substitute
                    temp = temp.replace(formula[0][2],newname)
                    br.append(labelsen(temp))
                    formula[2] = 0
                    didsomething = 1
                    break
                          
            if didsomething: continue
            
            # universals
            for formula in br:
                # collect all names
                names = namesinbr(br)
                # if all substitutions made, next
                if (formula[2] == names): continue
                
                if (formula[1] == 15): 
                    # strip quantifier
                    temp = formula[0][4:]
                    # collect all names
                    names = namesinbr(br)
                    # make every substitution not already substituted
                    for name in names:
                        if (name not in formula[2]):
                            subbed = temp.replace(formula[0][2],name)
                            br.append(labelsen(subbed))
                            formula[2].add(name)
                    
                    didsomething = 1
                    break
                          
            if didsomething: continue
            
            # branching--add extra element to encoding, to indicate branching
            for formula in br:
                # if done this formula, next
                if (formula[2] == 0): continue
                
                # negated conjunction
                if (formula[1] == 8): 
                    # new formulas
                    newL = '~'+formula[0][2:2+formula[2]]
                    newR = '~'+formula[0][3+formula[2]:-1]
                    # mark formula as done
                    formula[2] = 0
                    # create new branch
                    newbr = copy.deepcopy(br)
                    # add new formulas
                    br.append(labelsen(newR)+[0])
                    newbr.append(labelsen(newL))
                    # add new branch to tree
                    tr.append(newbr)
                    
                    didsomething = 1
                    break
                
                # disjunction
                if (formula[1] == 7): 
                    # new formulas
                    newL = formula[0][:formula[2]]
                    newR = formula[0][1+formula[2]:]
                    # mark formula as done
                    formula[2] = 0
                    # create new branch
                    newbr = copy.deepcopy(br)
                    # add new formulas
                    br.append(labelsen(newR)+[0])
                    newbr.append(labelsen(newL))
                    # add new branch to tree
                    tr.append(newbr)
                    
                    didsomething = 1
                    break
                
                # conditional
                if (formula[1] == 9): 
                    # new formulas
                    newL = '~'+formula[0][:formula[2]]
                    newR = formula[0][1+formula[2]:]
                    # mark formula as done
                    formula[2] = 0
                    # create new branch
                    newbr = copy.deepcopy(br)
                    # add new formulas
                    br.append(labelsen(newR)+[0])
                    newbr.append(labelsen(newL))
                    # add new branch to tree
                    tr.append(newbr)
                    
                    didsomething = 1
                    break
                
                # biconditional
                if (formula[1] == 10): 
                    # new formulas
                    newL1 = formula[0][:formula[2]]
                    newL2 = formula[0][1+formula[2]:]
                    newR1 = '~'+formula[0][:formula[2]]
                    newR2 = '~'+formula[0][1+formula[2]:]
                    # mark formula as done
                    formula[2] = 0
                    # create new branch
                    newbr = copy.deepcopy(br)
                    # add new formulas
                    br.append(labelsen(newR1)+[0])
                    br.append(labelsen(newR2))
                    newbr.append(labelsen(newL1))
                    newbr.append(labelsen(newL2))
                    # add new branch to tree
                    tr.append(newbr)
                    
                    didsomething = 1
                    break
                
                # negated biconditional
                if (formula[1] == 11): 
                    # new formulas
                    newL1 = '~'+formula[0][2:2+formula[2]]
                    newL2 = formula[0][3+formula[2]:-1]
                    newR1 = formula[0][2:2+formula[2]]
                    newR2 = '~'+formula[0][3+formula[2]:-1]
                    # mark formula as done
                    formula[2] = 0
                    # create new branch
                    newbr = copy.deepcopy(br)
                    # add new formulas
                    br.append(labelsen(newR1)+[0])
                    br.append(labelsen(newR2))
                    newbr.append(labelsen(newL1))
                    newbr.append(labelsen(newL2))
                    # add new branch to tree
                    tr.append(newbr)
                    
                    didsomething = 1
                    break
            
            if didsomething: continue
            
            # SI
            # collect all atomic formulas already present
            atoms = set([formula[0] for formula in br if formula[1] in {0,1,2}])
            # collect all identity sentences already present
            idens = set([formula[0] for formula in br if formula[1] == 2])
            
            # generate possible new atomics, replacing one at a time
            for iden in idens:
                for atom in atoms:
                    if iden[0] in atom: 
                        newguy = atom.replace(iden[0],iden[2],1)
                        if newguy not in atoms:
                            br.append(labelsen(newguy))
                            didsomething = 1
                    if iden[2] in atom: 
                        newguy = atom.replace(iden[2],iden[0],1)
                        if newguy not in atoms:
                            br.append(labelsen(newguy))
                            didsomething = 1
                          
            if didsomething: continue
    return tr

                
# read a model off an open branch
def readmodel(br):
    # read referents
    names = namesinbr(br)
    referents = {}
    ind = 1
    for name in names: referents[name] = ind; ind += 1
    
    # check for identity
    idens = set([formula[0] for formula in br if formula[1] == 2])
    for iden in idens:
        if iden[0] != iden[2]: referents[iden[0]] = referents[iden[2]]
    
    # collect referents into domain
    domain = set(referents.values())
    
    # collect atoms
    atoms = set([formula[0] for formula in br if formula[1] == 0])
    negatoms = set([formula[0] for formula in br if (formula[1] == 1 and formula[0][1].isupper())])
    
    # assign extensions
    exts = {}
    
    # negative atomic
    for atom in negatoms: exts[atom[1]] = set()
    
    # postiive atomic
    for atom in atoms:
        memberlist = []
        for char in atom[1:]: memberlist.append(str(referents[char]))
        member = '<' + ','.join(memberlist) + '>'
        if atom[0] in exts: exts[atom[0]].add(member)
        else: exts[atom[0]] = set([member])
    
    # collect propositions
    props = set([formula[0] for formula in br if formula[1] == 16])
    
    # return modelin the earlier format
    return [domain, referents, exts, props]
    

# read a model off a tree, nmodels is the max number of models user wants
def modelfromtree(tr, nmodels = 1):
    models = []
    
    for br in tr:
        if (br[-1] == 'OPEN BRANCH'): models.append(readmodel(br))
        if (len(models) == nmodels): break
    return models


# print a model
def printmod(model):
    print('Domain: ', model[0])
    if model[1]: print('Referents: ', model[1])
    if model[2]: print('Extension: ', model[2])
    if model[3]: print('Propositions: ', model[3])

    
# assess if tree is open
def openTr(tr):
    for br in tr:
        if (br[-1] == 'OPEN BRANCH'): return True
    return False
            

# compile tree into single list to facilitate printing
def compileTree(tr):
    # if tree has no branches, done
    if len(tr) == 1: return tr[0]
            
    # to map branching structure...
    # primary branch is where we're tracing, secondary is where we expect a branch
    # we keep a dict--structure--to map the branching structure of the tree
    # structure entries are sec: (pri, index)
    # when primary branch indicates divergence, we increment secondary and add to dict
    # when primary runs out, we increment primary; index resets to divergence point
    # when secondary == len(tr)-1 and we have divergence, we're done mapping
    
    str = {}
    idx = 0
    pri = 0
    sec = 1
    
    while True:
        # trace up to divergence or primary runs out
        while len(tr[pri][idx]) == 3: idx += 1
            
        # if diverge
        if len(tr[pri][idx]) == 4:
            # add divergence point 
            str[sec] = (pri, idx)
            # increment secondary
            sec += 1
            idx += 1
            # check if done
            if (sec >= len(tr)): break
        
        # if primary runs out
        if len(tr[pri][idx]) > 4:
            # increment primary
            pri += 1
            # find divergence point and set new index
            idx = str[pri][1]
    
    # merge branches at branching points
    for sec in range(len(tr)-1,0,-1):
        pri, idx = str[sec]
        brL = tr[pri][idx:]
        brR = tr[sec][idx:]
        tr[pri] = tr[pri][:idx] + [[brL, brR]]
    
    return tr[0]


# print compiled tree
def printCTree(tr, indent = 0):
    # print all but last
    preprint = '       '*indent
    for formula in tr[:-1]: print(preprint+post(formula[0]))
    
    # if no branch, print and finish
    if len(tr[-1])>2: print(preprint+tr[-1])
    # if branch, print branches recursively
    else: 
        print(preprint+'left branch: ')
        printCTree(tr[-1][0], indent = indent + 1)
        print(preprint+'right branch: ')
        printCTree(tr[-1][1], indent = indent + 1)
        

### Checking and converting wffs

This function takes a sentence as input and tells you whether that sentence is well-formed (according to infix notation).<br>
Outermost parenthesis can be omitted, but inner ones cannot. E.g., A&B&C is not a wff, (A&B)&C and (A&(B&C)) are
<br><br>
<i>Example</i><br>
Input sentence: (Ax)(P <-> Qxyz) ^ \~~a=b<br>
Output: well-formed

In [ ]:
sen = input("sentence: ")

sen = pre(sen)
if (iswff(sen) or iswff("("+sen+")")): print("well-formed")
else: print("not well-formed")

This function takes a sentence in infix notation and returns the equivalent sentence in prefix notation <br>
If the sentence is not well-formed the code throws an error
<br><br>
<i>Example</i><br>
Input sentence in prefix :(Ax)(P <-> Qxyz) ^ \~\~a=b<br>
Output: ^(Ax)<->PQxyz\~~=ab

In [ ]:
sen = senIn('sentence in infix')
print('converted to prefix:', post(inToPre(sen)))

This function takes a sentence as input and tells you whether that sentence is well-formed (according to prefix notation).<br>
<br>
<i>Example</i><br>
Input sentence: ^(Ax)<->PQxyz\~~=ab<br>
Output: well-formed

In [ ]:
sen = input("sentence: ")

sen = pre(sen)
if iswffP(sen): print("well-formed")
else: print("not well-formed")

This function takes a sentence in prefix notation and returns the equivalent sentence in infix notation <br>
If the sentence is not well-formed the code says so and gives no output
<br><br>
<i>Example</i><br>
Input sentence in prefix: ^(Ax)<->PQxyz\~\~=ab<br>
Output: ((Ax)(P<->Qxyz)^\~\~a=b)

In [ ]:
sen = input("sentence in prefix: ")

sen = pre(sen)
if not iswffP(sen): print('sentence wasn\'t well formed')
else: print('converted to infix: ',post(preToIn(sen)))

### Models

This function checks whether a model passes some basic tests
- domains can't be empty
- names in lowercase letters
- predicates and propositions in uppercase letters
- predicate arity has to be consistent

For input, 
domain: type elements separated by commas<br>
referents: colon between name and referent, commas between entries<br>
extensions: colon between predicate and extension, commas between entries, <> for relations, {} for empty set<br>
propositions: enter the symbols for true propositions, separated by commas

if a section of the model is blank, leave the input blank

If verbose (parameter below) is set to True, and something is wrong with the model, the code will say what


<i>Example</i><br>
Input: <br>
basic propositions: A,B,C<br>
domain: alice, bob, charlie<br>
referents: a: alice, b: bob, c: bob<br>
extensions: F: {}, G: {<alice,charlie>}, H: {alice, bob}<br>
Output: ok

In [ ]:
verbose = True
model = modin()

if modcheck(model, verbose=verbose): print('ok')
elif not verbose: print('something error model')

This function takes a model and sentence as input, then tells you whether the sentence is true in that model<br>
If there's an error in the model the code throws an exception<br>
If verbose (parameter below) is set to True, the code will also explain how we arrive at the answer

<i>Example</i><br>
Input<br>
basic propositions: A,B,C<br>
domain: alice, bob, charlie<br>
referents: a: alice, b: bob, c: bob<br>
extensions: F: {}, G: {<alice,charlie>}, H: {alice, bob}<br>
sentence: (Ex)Fx<br><br>
Output<br>
(Ex)Fx is false because Fx is false under all substitutions of x<br>
&emsp;When x is bob...<br>
&emsp;&emsp;Fx is False because ['bob'] is not in the extension of F<br>
&emsp;When x is alice...<br>
&emsp;&emsp;Fx is False because ['alice'] is not in the extension of F<br>
&emsp;When x is charlie...<br>
&emsp;&emsp;Fx is False because ['charlie'] is not in the extension of F<br>
So the sentence is False<br>

In [ ]:
verbose = True

model = modin()
if not modcheck(model, verbose = False): raise Exception('some error in model')

# are there more sentences
more = 1

while more != '0': 
    sen = input("sentence: ")
    print('\n')
    sen = pre(sen)

    if not (iswff(sen) or iswff("("+sen+")")): raise Exception('sentence wasn\'t well formed')
    else: print(f'So the sentence is {eval(sen,model,verbose=verbose)}')
    
    more = input('type 0 if done ')

### Trees

This function takes a single sentence as input and returns the completed tree of that sentence<br>
If the tree is open, the code also prints a model of that sentence<br>
The num_models parameter below tells the code how many models you want, the code will print models up to that number
<br><br>
<i>Example</i><br>
Input sentence: sentence:  :(Ex)(Ay)x=y

Output <br>
Tree:<br>
(Ex)(Ay)x=y<br>
(Ay)a=y<br>
a=a<br>
OPEN BRANCH<br>

Model:<br>
Domain:  {1}<br>
Referents:  {'a': 1}

In [ ]:
num_models = 1

sen = senIn('sentence: ')

tr = finish([[labelsen(sen)]])
isopen = openTr(tr)
if isopen: models = modelfromtree(tr, nmodels=num_models)

print('\n------')
print('Tree')
print('------')
printCTree(compileTree(tr))

if isopen: 
    for i in range(min(num_models, min(num_models, len(models)))):
        print('\n------')
        print(f'Model {i+1}')
        print('------')
        printmod(models[i])

This function runs a satisfiability test on a set of sentences. Input sentences one by one and leave the input blank when you are done<br>
If tree is open, this code prints models satisfying the set

In [ ]:
num_models = 1

br = labSetIn('sentence')
tr = finish([br])
isopen = openTr(tr)
if isopen: 
    print('\nSet is satisfiable; tree is open')
    models = modelfromtree(tr, nmodels=num_models)
else: print('\nSet is not satisfiable; tree is closed')
    
print('\n------')
print('Tree')
print('------')
printCTree(compileTree(tr))

if isopen: 
    for i in range(min(num_models, min(num_models, len(models)))):
        print('\n------')
        print(f'Model {i+1}')
        print('------')
        printmod(models[i])

This function runs a validity test on an argument. Input premises one by one and leave the input blank when you are done<br>
Then input the conclusion (no need to negate for the tree test, this code adds the negation for you)<br>
If tree is open, this code prints countermodels

In [ ]:
num_models = 1

br = labSetIn('premise')
sen = senIn('conclusion')
if iswff(sen): sen = '~'+sen
else: sen = '~(' + sen + ')'

br.append(labelsen(sen))
tr = finish([br])
isopen = openTr(tr)
if isopen: 
    print('\nArgument is invalid; tree is open')
    models = modelfromtree(tr, nmodels=num_models)
else: print('\nArgument is valid; tree is closed')
    
print('\n------')
print('Tree')
print('------')
printCTree(compileTree(tr))

if isopen: 
    for i in range(min(num_models, min(num_models, len(models)))):
        print('\n------')
        print(f'Countermodel {i+1}')
        print('------')
        printmod(models[i])

This function runs a contradiction test on a sentence<br>
If tree is open, this code prints a model to make the sentence true

In [ ]:
num_models = 1

br = senIn('sentence')
tr = finish([[labelsen(br)]])
isopen = openTr(tr)
if isopen: 
    print('\nNot a contradiction; tree is open')
    models = modelfromtree(tr, nmodels=num_models)
else: print('\nContradiction; tree is closed')
    
print('\n------')
print('Tree')
print('------')
printCTree(compileTree(tr))

if isopen: 
    for i in range(min(num_models, min(num_models, len(models)))):
        print('\n------')
        print(f'Model to make sentence true: model {i+1}')
        print('------')
        printmod(models[i])

This function runs a tautology test on a sentence<br>
If tree is open, this code prints a model to make the sentence false

In [ ]:
num_models = 3

sen = senIn('sentence')
if iswff(sen): sen = '~'+sen
else: sen = '~(' + sen + ')'
    
tr = finish([[labelsen(sen)]])
isopen = openTr(tr)
if isopen: 
    print('\nNot a tautology; tree is open')
    models = modelfromtree(tr, nmodels=num_models)
else: print('\nTautology; tree is closed')
    
print('\n------')
print('Tree')
print('------')
printCTree(compileTree(tr))

if isopen: 
    for i in range(min(num_models, min(num_models, len(models)))):
        print('\n------')
        print(f'Model to make sentence false: model {i+1}')
        print('------')
        printmod(models[i])

This function runs an equivalence test on a pair of sentences<br>
Input the two sentences one at a time<br>
If tree is open, this code prints a model to make the two sentences differ

In [ ]:
num_models = 1

sen1 = senIn('first sentence')
if not iswff(sen1): sen1 = '(' + sen1 + ')'

sen2 = senIn('second sentence')
if not iswff(sen2): sen2 = '(' + sen2 + ')'

mainsen = '~(' + sen1 + '<' + sen2 + ')'
    
tr = finish([[labelsen(mainsen)]])
isopen = openTr(tr)
if isopen: 
    print('\nNot equivalent; tree is open')
    models = modelfromtree(tr, nmodels=num_models)
else: print('\nEquivalent; tree is closed')
    
print('\n------')
print('Tree')
print('------')
printCTree(compileTree(tr))

if isopen: 
    for i in range(min(num_models, min(num_models, len(models)))):
        print('\n------')
        print(f'Model to make truth values differ: model {i+1}')
        print('------')
        printmod(models[i])